In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import ndcg_score,dcg_score

In [ ]:

def eval(pred_path,true_path):

    '''
    Reads the Predicted Recommendations and Compares it with the Actual Purchases in the Testing Period
    Arguments : 
        pred_path - Path to the File containing the Generated Recommendations
        true_path - Path to the File containing the Actual Purchases during the testing week
    
    '''
    rec = pd.read_csv(pred_path)
    true = pd.read_csv(true_path)
    rec = rec[rec.customer_id.isin(true.customer_id)]
    df = pd.DataFrame(true.groupby("customer_id")['article_id'].apply(list))
    df['customer_id']  = df.index
    df = df.rename(columns = {'article_id':'relevant_items'})
    df = df.reset_index(drop=True)
    whole_data = rec.merge(df,on="customer_id")

    k = []

    for i in whole_data['article_id']:
      k.append(i.strip('][').split(','))

    l = []

    for i in whole_data['relevant_items']:
      #l.append(i)
      l.append(str(i).strip('][').split())
      
    return (np.mean([mapk(l,k,k=i) for i in range(1,13) ]))

In [ ]:
def apk(actual, predicted, k=12):
    # Computes Average Precision at K
    if len(predicted)>k:
        predicted = predicted[:k]
    
    score = 0.0
    num_hits = 0.0
    #predicted = predicted.split()
    #actual = actual.split()
    c=0
    for i,p in enumerate(predicted):
        if int(p) in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
        
    return score / min(len(actual), k)

def mapk(actual, predicted, k=12):
    #Computes Mean Average Precision at K 
 
    return np.mean([apk(a,p,k) for a,p in zip(actual,predicted)])

In [ ]:
def pred_ndcg_score(pred,actual):

  '''
    Reads both the predicted and the actual dataset and adds the ndcg score for each customer id and then take average of it
    Arguments:
        pred - predictions file generated by the model
        actual - test file containing last week of purchases made by the customers
    
    '''
  
  pred = pred[pred.customer_id.isin(actual.customer_id)]
  pred=pred.reset_index()
  
  
  pred = pred.rename(columns = {'article_id':'relevant_items'})
  pred = pred.reset_index(drop=True)
  whole_data = actual.merge(pred,on="customer_id")
  k = []

  for i in whole_data['article_id']:
    k.append([int(j) for j in str(i).strip('][').split()])

  l = []

  for i in whole_data['relevant_items']:
    
    l.append([int(j) for j in str(i).strip('][').split(',')])
  probs=[]
  for i in whole_data['probability']:
    
    probs.append([float(j) for j in str(i).strip('][').split(',')])
  c,tot=0,0
  for p,a,prob in zip(l,k,probs):
    for i,j in enumerate(p):
      if j in a:
        p[i]=1
      else:
        p[i]=0
      
    
    tot+=ndcg_score([p],[prob])
    c+=1
  return tot

In [ ]:
dlrm_preds=pd.read_csv("predictions_DLRM.csv")
content_based_preds=pd.read_csv("predictions_content_filtering.csv")
collab_based_preds=pd.read_csv("predictions_collab_filtering.csv")
test=pd.read_csv("transactions_test_subset.csv")


In [ ]:
test

,customer_id,article_id
0,00356af493f63b3d5c800d66e5ea35e255e3e602c12ec6...,"[592383002, 288825017, 288825018, 785188002, 7..."
1,007006426cbb9f97cc1a13b6e674429f97143ab42ae363...,"[592383002, 288825017, 288825018, 645064001, 7..."
2,00c28643e072e316684339c10636fe8b3c3b0628d70979...,"[755187001, 712990002, 857750002, 712990001, 5..."
3,01611cbd05eff1171095ab89759f0f3bc71d7915e5f2eb...,"[592383002, 288825017, 288825018, 785188002, 7..."
4,01639ee2305ab5e99f32a84c49d4ea7cb2c48b400f0c50...,"[592383002, 785188002, 288825017, 288825018, 6..."
...,...,...
943,ff0ea7337ce95db29cf03e0ebead119e827c23b30205ed...,"[755187001, 712990002, 645064001, 592383002, 6..."
944,ff7fabb47d14d21589c7f534ee150ab010e8ace28ec5d1...,"[592383002, 785188002, 288825017, 288825018, 6..."
945,ffa4121ec19735cbad2cda310778873ab7a48963ed7ebe...,"[592383002, 288825017, 288825018, 785188002, 5..."
946,ffa938ef7d86f1d9793605bc3fc2a6736fb44b902984df...,"[592383002, 288825017, 785188002, 288825018, 6..."


In [1]:
print("MAP@12 score using content based filtering is", eval(content_based_preds,test))
print("MAP@12 score using Item-item collaborative filtering is", eval(collab_based_preds,test))
print("MAP@12 score using DLRM is", eval(dlrm_preds,test))

MAP@12 score using content based filtering is 0.02135
MAP@12 score using Item-item collaborative filtering is 0.02204
MAP@12 score using DLRM is 0.03123


In [3]:
print("NDCG score of content based filtering is", pred_ndcg_score(content_based_preds,test))
print("NDCG score of Item-item collaborative filtering is", pred_ndcg_score(collab_based_preds,test))
print("NDCG score of DLRM is", pred_ndcg_score(dlrm_preds,test))

NDCG score of content based filtering is 0.5217
NDCG score of Item-item collaborative filtering is 0.5603
NDCG score of DLRM is 0.7092
